In [53]:
from functools import reduce
from operator import mul
from multiprocessing import Pool, cpu_count
from pathos.multiprocessing import ProcessingPool as Pool
import pandas as pd
import numpy as np
import time

In [54]:
import os
os.cpu_count()

8

1. Write a version of curried function c_5 using lambda syntax/
Напишіть версію функції c_5, використовуючи лямбда-синтаксис.

In [55]:
c_5 = lambda a: lambda b: lambda c: lambda d: lambda e: a + b + c + d + e

print(c_5(1)(2)(3)(4)(5))

15


2. Write a program calculating a factorial using parallelisation and map-reduce/
Напишіть програму, яка обчислює факторіал, використовуючи паралелізацію та map-reduce.

In [56]:
def factorial(n):
    return reduce(lambda x, y: x*y, range(1, n+1), 1)

n=5

pool = ProcessPool()
result = pool.map(factorial, [n])

print(result[0]) 

120


In [67]:
def chunked_factorial(n, chunks=4):
    ranges = []
    step = n // chunks
    for i in range(chunks):
        start = i * step + 1
        end = (i + 1) * step if i < chunks - 1 else n
        ranges.append(range(start, end + 1))
    return ranges

def partial_product(r):
    return reduce(mul, r, 1)

def parallel_factorial(n, chunks=None):
    if chunks is None:
        chunks = cpu_count()
    ranges = chunked_factorial(n, chunks)
    with Pool(chunks) as pool:
        partial_results = pool.map(partial_product, ranges)
    return reduce(mul, partial_results, 1)

print("5! =", parallel_factorial(5))   # 120
print("10! =", parallel_factorial(10)) # 3628800

5! = 120
10! = 3628800


3. Write a parallelised version of reduce (hint: use chunking)/Напишіть паралелізовану версію reduce (підказка: використовуйте фрагментацію).

In [68]:
def parallel_sum(lst, n_chunks=2):
    chunks = np.array_split(lst, n_chunks)
    
    with Pool() as pool:
        partial_sums = pool.map(sum, chunks)
        
    return sum(partial_sums)

int_list = list(range(1, 101))
print(parallel_sum(int_list, n_chunks=4))  # 5050

5050


4. Compare performance of Numpy vectorization vs parallel map on a really large array of integers./
Порівняйте продуктивність векторизації Numpy та паралельного відображення на дійсно великому масиві цілих чисел.

In [59]:
start = time.time()

np_array = np.arange(1, 1000000)

vectorized_fn = np.vectorize(lambda x: x*x)

result = vectorized_fn(np_array)
#print(result)

end = time.time()
print("np.vectorize time:", end - start)

np.vectorize time: 0.18257617950439453


In [60]:
start = time.time()
res_numpy = np_array * np_array
end = time.time()

print("NumPy vectorization time:", end - start)

NumPy vectorization time: 0.008358955383300781


In [61]:
from pathos.multiprocessing import ProcessPool

start = time.time()
pool = ProcessPool()
result = pool.map(lambda x: x*x, np_array)
end = time.time()

print("Parallel map with Pathos:", end - start)

Parallel map with Pathos: 13.01998782157898


In [62]:
"""
def sqr(x):
    return x*x

np_array = np.arange(1, 100000)

n_processes = 4
chunksize = max(1, len(np_array) // (n_processes * 4))

start = time.time()
with Pool(n_processes) as pool:
    result = pool.map(sqr, np_array, chunksize=chunksize)
end = time.time()

print("Parallel map time:", end - start)
"""

'\ndef sqr(x):\n    return x*x\n\nnp_array = np.arange(1, 100000)\n\nn_processes = 4\nchunksize = max(1, len(np_array) // (n_processes * 4))\n\nstart = time.time()\nwith Pool(n_processes) as pool:\n    result = pool.map(sqr, np_array, chunksize=chunksize)\nend = time.time()\n\nprint("Parallel map time:", end - start)\n'

5. Using computations from previous labs, apply chunking approach to df.apply(...) using parallel map./ Використовуючи обчислення з попередніх лабораторних робіт, застосуйте підхід фрагментації до df.apply(...) з використанням паралельного відображення.

In [63]:
df = pd.read_csv("winemag-data-130k-v2.csv")
df["price"] = df["price"].fillna(df["price"].mean())
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,35.363389,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.000000,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.000000,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [64]:
def points_per_dollar_py(row):
    return row["points"] / row["price"]

In [65]:
"""
def compute_chunk(chunk):
    return chunk.apply(points_per_dollar_py, axis=1)

def parallel_compute(df, n_chunks=None):
    if n_chunks is None:
        n_chunks = cpu_count()  # автоматично підбираємо кількість процесів

    chunk_size = len(df) // n_chunks
    chunks = [df.iloc[i*chunk_size:(i+1)*chunk_size] for i in range(n_chunks)]

    with Pool(n_chunks) as pool:
        results = pool.map(compute_chunk, chunks)

    return pd.concat(results)
"""

%timeit df.apply(points_per_dollar_py, axis=1)
#%timeit parallel_compute(df.head(10000), 4)


349 ms ± 3.98 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [66]:
def points_per_dollar_py(row):
    return row["points"] / row["price"]

def compute_chunk(chunk):
    return chunk.apply(points_per_dollar_py, axis=1)

def parallel_compute(df, n_chunks=None):
    if n_chunks is None:
        n_chunks = cpu_count()

    chunks = np.array_split(df, n_chunks)

    with Pool(n_chunks) as pool:
        results = pool.map(compute_chunk, chunks)

    return pd.concat(results)

%timeit parallel_compute(df.head(10000), 4)  

78 ms ± 3.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
